In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

* 분석 결과 : casual와 registered 제외됨 V
* 분석 결과 : 파생 피처 추가 -> 년 월 일 시 분 초 요일  V
* 분석 결과 : datetime 피처 제거 -> 데이터를 구분하는 역할이므로 타겟값에 영향이 없음 
* 분석 결과 : month 제거 V
* 분석결과 : 타겟값을 count가 아닌 log(count)로 변환하여 사용!!!, 예측후 지수변환하여 count 계산 np.exp() 활용!! V
* 분석 결과 : day, minute, second 피처 제거 가능 V
* 분석 결과 : weather == 4 인 데이터 이상치로 삭제 V
* 분석 결과 : windspeed 피처 삭제V

In [3]:
# 피쳐 엔지니어링
from datetime import datetime
import calendar

train['year'] = train.datetime.map(lambda x: str(datetime.strptime(x, '%Y-%m-%d %H:%M:%S').year).zfill(4))
#train['month'] = train.datetime.map(lambda x: str(datetime.strptime(x, '%Y-%m-%d %H:%M:%S').month).zfill(2))
#train['day'] = train.datetime.map(lambda x: str(datetime.strptime(x, '%Y-%m-%d %H:%M:%S').day).zfill(2))
train['hour'] = train.datetime.map(lambda x: str(datetime.strptime(x, '%Y-%m-%d %H:%M:%S').hour).zfill(2))
#train['minute'] = train.datetime.map(lambda x: str(datetime.strptime(x, '%Y-%m-%d %H:%M:%S').minute).zfill(2))
#train['second'] = train.datetime.map(lambda x: str(datetime.strptime(x, '%Y-%m-%d %H:%M:%S').second).zfill(2))
train['weekday'] = train.datetime.map(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S').weekday()) # ML 수행시
train

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,hour,weekday
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3,13,16,2011,00,5
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8,32,40,2011,01,5
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5,27,32,2011,02,5
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3,10,13,2011,03,5
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0,1,1,2011,04,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10881,2012-12-19 19:00:00,4,0,1,1,15.58,19.695,50,26.0027,7,329,336,2012,19,2
10882,2012-12-19 20:00:00,4,0,1,1,14.76,17.425,57,15.0013,10,231,241,2012,20,2
10883,2012-12-19 21:00:00,4,0,1,1,13.94,15.910,61,15.0013,4,164,168,2012,21,2
10884,2012-12-19 22:00:00,4,0,1,1,13.94,17.425,61,6.0032,12,117,129,2012,22,2


In [4]:
# 피쳐 엔지니어링
from datetime import datetime
import calendar

test['year'] = test.datetime.map(lambda x: str(datetime.strptime(x, '%Y-%m-%d %H:%M:%S').year).zfill(4))
#train['month'] = train.datetime.map(lambda x: str(datetime.strptime(x, '%Y-%m-%d %H:%M:%S').month).zfill(2))
#train['day'] = train.datetime.map(lambda x: str(datetime.strptime(x, '%Y-%m-%d %H:%M:%S').day).zfill(2))
test['hour'] = test.datetime.map(lambda x: str(datetime.strptime(x, '%Y-%m-%d %H:%M:%S').hour).zfill(2))
#train['minute'] = train.datetime.map(lambda x: str(datetime.strptime(x, '%Y-%m-%d %H:%M:%S').minute).zfill(2))
#train['second'] = train.datetime.map(lambda x: str(datetime.strptime(x, '%Y-%m-%d %H:%M:%S').second).zfill(2))
test['weekday'] = test.datetime.map(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S').weekday()) # ML 수행시
test

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,year,hour,weekday
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027,2011,00,3
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000,2011,01,3
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000,2011,02,3
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014,2011,03,3
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014,2011,04,3
...,...,...,...,...,...,...,...,...,...,...,...,...
6488,2012-12-31 19:00:00,1,0,1,2,10.66,12.880,60,11.0014,2012,19,0
6489,2012-12-31 20:00:00,1,0,1,2,10.66,12.880,60,11.0014,2012,20,0
6490,2012-12-31 21:00:00,1,0,1,1,10.66,12.880,60,11.0014,2012,21,0
6491,2012-12-31 22:00:00,1,0,1,1,10.66,13.635,56,8.9981,2012,22,0


In [5]:
train = train.drop(columns=['datetime', 'windspeed', 'casual', 'registered'])

In [6]:
test = test.drop(columns=['datetime', 'windspeed'])

In [7]:
train['count_logit'] = train['count'].map(lambda x: np.log(x))

In [8]:
train = train[train['weather']!=4]

In [9]:
train = train.loc[:,['season', 'holiday', 'workingday', 'weather', 'temp', 'atemp', 'humidity', 'year', 'hour', 'weekday', 'count_logit']]

In [10]:
train.shape, test.shape

((10885, 11), (6493, 10))

In [11]:
def rmsle(y_true, y_pred, convertExp=True):
    if convertExp:
        y_true = np.exp(y_true)
        y_pred = np.exp(y_pred)
    log_ture = np.nan_to_num(np.log(y_true+1))
    log_pred = np.nan_to_num(np.log(y_pred+1))
    
    output = np.sqrt(np.mean((log_ture - log_pred)**2))
    return output

In [12]:
train_X = train.loc[:,['season', 'holiday', 'workingday', 'weather', 'temp', 'atemp', 'humidity', 'year', 'hour', 'weekday']]
train_y = train.count_logit

In [13]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
model.fit(train_X, train_y)
model.score(train_X, train_y)

0.992666161339162

In [14]:
pred = model.predict(test)

In [15]:
submission=pd.read_csv('./data/sampleSubmission.csv')
submission['count'] = np.exp(pred)
# submission.to_csv('./submisstion.csv',index=False)

In [16]:
help(RandomForestRegressor)

Help on class RandomForestRegressor in module sklearn.ensemble._forest:

class RandomForestRegressor(ForestRegressor)
 |  RandomForestRegressor(n_estimators=100, *, criterion='squared_error', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, ccp_alpha=0.0, max_samples=None)
 |  
 |  A random forest regressor.
 |  
 |  A random forest is a meta estimator that fits a number of classifying
 |  decision trees on various sub-samples of the dataset and uses averaging
 |  to improve the predictive accuracy and control over-fitting.
 |  The sub-sample size is controlled with the `max_samples` parameter if
 |  `bootstrap=True` (default), otherwise the whole dataset is used to build
 |  each tree.
 |  
 |  Read more in the :ref:`User Guide <forest>`.
 |  
 |  Parameters
 |  ----------
 |  n_estimato

In [21]:
from sklearn import metrics
rmsle_scorer = metrics.make_scorer(rmsle, greater_is_better=False)

In [22]:
from sklearn.model_selection import GridSearchCV
param_grid ={'min_samples_leaf': range(1,50,5), 'min_samples_split': range(2, 50, 5)}
model = RandomForestRegressor()
grid_model = GridSearchCV(model, param_grid=param_grid, scoring=rmsle_scorer, cv=5)
grid_model.fit(train_X, train_y)

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'min_samples_leaf': range(1, 50, 5),
                         'min_samples_split': range(2, 50, 5)},
             scoring=make_scorer(rmsle, greater_is_better=False))

In [26]:
pred = grid_model.best_estimator_.predict(test)

In [28]:
submission=pd.read_csv('./data/sampleSubmission.csv')
submission['count'] = np.exp(pred)
submission.to_csv('./submisstion_grid.csv',index=False)